In [1]:
import pandas as pd
import numpy as np

## Preview data

In [3]:
busi = pd.read_csv('business_data_subset.csv')
rev = pd.read_csv('reviews_data_subset.csv', parse_dates=['date'])
user = pd.read_csv('user_df.csv', low_memory=False)

In [4]:
# business_data
busi.head(1)

,business_id,business_name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."


In [5]:
# review_data
rev.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03


In [6]:
# user_data
user.head(1)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180


## Concat review_df & business_df

In [7]:
rev_ = rev[['review_id', 'user_id', 'business_id', 'stars', 'text', 'date']]
busi_ = busi[['business_id', 'business_name', 'city', 'state', 'categories']]

# add business info onto each review
rev_busi_ = rev_.merge(busi_, how='outer', on='business_id')
rev_busi_ = rev_busi_.reset_index(drop=True)
rev_busi_.head(1)

,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"


In [10]:
rev_busi_.info() # 3,000,000 reviews

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2043247 entries, 0 to 2043246
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   review_id      object        
 1   user_id        object        
 2   business_id    object        
 3   stars          float64       
 4   text           object        
 5   date           datetime64[ns]
 6   business_name  object        
 7   city           object        
 8   state          object        
 9   categories     object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 155.9+ MB


## Filter based on City

In [11]:
# only maintain reviews that meets city = 'Phoenix'
rev_busi_Phl = rev_busi_[rev_busi_['city'] == 'Philadelphia']
rev_busi_Phl = rev_busi_Phl.reset_index(drop=True)

rev_busi_Phl.head(1) # 500,000 reviews

,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
0,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"


In [12]:
rev_busi_Phl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687289 entries, 0 to 687288
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   review_id      687289 non-null  object        
 1   user_id        687289 non-null  object        
 2   business_id    687289 non-null  object        
 3   stars          687289 non-null  float64       
 4   text           687289 non-null  object        
 5   date           687289 non-null  datetime64[ns]
 6   business_name  687289 non-null  object        
 7   city           687289 non-null  object        
 8   state          687289 non-null  object        
 9   categories     687289 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 52.4+ MB


THUS, we have around 600k reviews in phillydalphia


## Filter reviews: only keep the latest review for each user on each business

In [16]:
user_busi_groupby = rev_busi_Phl.groupby(['user_id', 'business_id'])
user_busi_groupby.head()
review_id_lst = []
for group_name, group_df in user_busi_groupby:
    group_df.sort_values(by='date')
    group_df.reset_index(inplace=True, drop=True)
    review_id_lst.append(group_df.iloc[-1,0]) # the latest review by each user on each business , 

In [17]:
len(review_id_lst)

665755

HENCE WE HAVE AROUND 600K MOST RECENT REVIEWS. REVIEW PER USER PER BUSINESS.
1. WE FILTER DOWN THE REVIEW DATA SET TO ONLY INCLUDE THESE RECENT REVIEWS.



In [18]:
review_id_lst = set(review_id_lst)
rev_busi_Phl = rev_busi_Phl[rev_busi_Phl['review_id'].isin(review_id_lst)]
len(rev_busi_Phl) 

665755

## Filter users & business based on reviews' counts

In [23]:
def review_filters(data, filtered_var, base_var, threshold):
    '''
    only keep the values of filter_var that meets:
    filter_var[base_var].sum() >= threshold
    '''
    count_df = data[[filtered_var, base_var]].groupby(filtered_var).count()
    valid_id = count_df[count_df[base_var]>=threshold].index.tolist()
    valid_id = set(valid_id)
    data = data[rev_busi_Phl[filtered_var].isin(valid_id)]
    return data

In [24]:
# user_id['review_id'].sum() >= 5,
# threshold for the users, as we are usiing very few users for now, origiinally it was kept as 5.
rev_busi_Phl = review_filters(rev_busi_Phl, 'user_id', 'review_id', 5)  
rev_busi_Phl.shape

(394068, 10)

WE filter down from 600k reviews to almost 394k reviews by keeping only users who have put >= 5 ratings

In [26]:
# business_id['review_id'].sum() >= 30
rev_busi_Phl = review_filters(rev_busi_Phl, 'business_id', 'review_id', 30)
rev_busi_Phl.shape


(357582, 10)

In [27]:
rev_busi_Phl.describe() 

,stars,date
count,357582.000000,357582
mean,3.877499,2015-05-18 07:37:57.839491840
min,1.000000,2005-05-25 01:12:54
25%,3.000000,2012-10-06 02:53:03.500000
50%,4.000000,2015-07-05 03:34:34
75%,5.000000,2018-02-21 15:01:44.500000
max,5.000000,2022-01-19 19:26:37
std,1.133976,NaN


## Output the filtered csv

In [28]:
rev_busi_Phl.to_csv('filtered_reviews_in_Philly.csv')

In [29]:
rev_busi_Phl.head()

,review_id,user_id,business_id,stars,text,date,business_name,city,state,categories
5,l-bH0C-PSXmVs-vQqMfJ1Q,toFm0jWZJKqsOmHE7kMbRA,kxX2SOes4o-D3ZQBkiMRfA,5.0,A very neat little hole in the wall. Very clea...,2015-06-21 01:11:14,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
7,E2rc2AjhzYxtoVQigrTpvg,DVMopL-MS3_03qMW0Dxa1A,kxX2SOes4o-D3ZQBkiMRfA,4.0,I've been here several times. The price is rea...,2014-11-13 20:50:21,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
8,3mtQ9nqWoyZjgidMQR6uxg,rmYqxdycak7SL9vY7xE21A,kxX2SOes4o-D3ZQBkiMRfA,5.0,"I found this place through Yelp reviews, as I'...",2014-12-18 17:39:44,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
11,VmM83BrA_db6jd1IwhNALw,JXB91quCyabpRX0fwP-RDA,kxX2SOes4o-D3ZQBkiMRfA,5.0,"I love the nice friendly service,great Indian ...",2015-09-10 17:33:00,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
13,Ol2Y4ficEqARYj-FfBG0fg,UzzWl_rQkjAi1iRzd-e-ZA,kxX2SOes4o-D3ZQBkiMRfA,5.0,Came here with my husband one night after he c...,2016-02-14 19:09:57,Zaika,Philadelphia,PA,"Halal, Pakistani, Restaurants, Indian"
